# Recover `MetricLogger.save_log` from raw `run.log`

This notebook scans runs under `deep_training/` and `tabular_training/`, parses metric print lines from `run.log`, and reconstructs log files with the same schema as `MetricLogger.save_log`.

In [8]:
from __future__ import annotations

import re
from dataclasses import dataclass
from pathlib import Path
from typing import Iterable

import pandas as pd

In [9]:
# Folder containing deep_training/ and tabular_training/
BASE_DIR = Path.cwd()

# If notebook is started elsewhere, point explicitly to your experiments folder:
# BASE_DIR = Path('/teamspace/studios/this_studio/Mario_RL/experiments ')

TRAINING_DIRS = [BASE_DIR / 'deep_training', BASE_DIR / 'tabular_training']

HEADER = (
    f"{'Episode':>8}{'Step':>8}{'Epsilon':>10}{'MeanReward':>15}"
    f"{'MeanLength':>15}{'MeanLoss':>15}{'MeanQValue':>15}"
    f"{'TimeDelta':>15}{'Time':>20}\n"
)

FLOAT_TOKEN = r'(?:nan|inf|-inf|[-+]?\d*\.?\d+(?:[eE][-+]?\d+)?)'
METRIC_LINE_RE = re.compile(
    rf'Episode (?P<episode>\d+) - '
    rf'Step (?P<step>\d+) - '
    rf'Epsilon (?P<epsilon>{FLOAT_TOKEN}) - '
    rf'Mean Reward (?P<mean_reward>{FLOAT_TOKEN}) - '
    rf'Mean Length (?P<mean_length>{FLOAT_TOKEN}) - '
    rf'Mean Loss (?P<mean_loss>{FLOAT_TOKEN}) - '
    rf'Time Delta (?P<time_delta>{FLOAT_TOKEN}) - '
    rf'Time (?P<time>[0-9T:-]+)'
)


def _to_float(token: str) -> float:
    token = token.strip().lower()
    if token == 'nan':
        return float('nan')
    if token == 'inf':
        return float('inf')
    if token == '-inf':
        return float('-inf')
    return float(token)


def _format_like_metric_logger(row: dict) -> str:
    # Mirrors experiments/metric_logger.py::record() row formatting
    return (
        f"{int(row['episode']):8d}{int(row['step']):8d}{float(row['epsilon']):10.3f}"
        f"{float(row['mean_reward']):15.3f}{float(row['mean_length']):15.3f}{float(row['mean_loss']):15.3f}"
        f"{float(row['time_delta']):15.3f}"
        f"{row['time']:>20}\n"
    )


def discover_run_logs(training_dirs: Iterable[Path]) -> list[Path]:
    run_logs: list[Path] = []
    for training_dir in training_dirs:
        if not training_dir.exists():
            continue
        run_logs.extend(sorted(training_dir.glob('*/run.log')))
    return run_logs


@dataclass
class RecoveryResult:
    run_name: str
    run_log: Path
    recovered_log: Path
    rows_recovered: int


def recover_metric_log_from_run_log(run_log_path: Path, output_name: str = 'log.recovered') -> RecoveryResult:
    recovered_rows: list[dict] = []

    for line in run_log_path.read_text(encoding='utf-8', errors='ignore').splitlines():
        for match in METRIC_LINE_RE.finditer(line):
            groups = match.groupdict()
            recovered_rows.append({
                'episode': int(groups['episode']),
                'step': int(groups['step']),
                'epsilon': _to_float(groups['epsilon']),
                'mean_reward': _to_float(groups['mean_reward']),
                'mean_length': _to_float(groups['mean_length']),
                'mean_loss': _to_float(groups['mean_loss']),
                'time_delta': _to_float(groups['time_delta']),
                'time': groups['time'],
            })

    logs_dir = run_log_path.parent / 'logs'
    logs_dir.mkdir(parents=True, exist_ok=True)
    recovered_log_path = logs_dir / output_name

    with recovered_log_path.open('w', encoding='utf-8') as f:
        f.write(HEADER)
        for row in recovered_rows:
            f.write(_format_like_metric_logger(row))

    return RecoveryResult(
        run_name=run_log_path.parent.name,
        run_log=run_log_path,
        recovered_log=recovered_log_path,
        rows_recovered=len(recovered_rows),
    )

In [10]:
run_logs = discover_run_logs(TRAINING_DIRS)
print(f'Found {len(run_logs)} run.log file(s).')
for path in run_logs:
    print('-', path)

Found 3 run.log file(s).
- /teamspace/studios/this_studio/Mario_RL/experiments /deep_training/double_q_run_17_feb/run.log
- /teamspace/studios/this_studio/Mario_RL/experiments /tabular_training/q_learning_17_feb/run.log
- /teamspace/studios/this_studio/Mario_RL/experiments /tabular_training/tabular_training_sarsa_1/run.log


In [11]:
results = [recover_metric_log_from_run_log(run_log) for run_log in run_logs]

summary_df = pd.DataFrame([
    {
        'run_name': r.run_name,
        'run_log': str(r.run_log),
        'recovered_log': str(r.recovered_log),
        'rows_recovered': r.rows_recovered,
    }
    for r in results
])

summary_df

,run_name,run_log,recovered_log,rows_recovered
0,double_q_run_17_feb,/teamspace/studios/this_studio/Mario_RL/experi...,/teamspace/studios/this_studio/Mario_RL/experi...,47
1,q_learning_17_feb,/teamspace/studios/this_studio/Mario_RL/experi...,/teamspace/studios/this_studio/Mario_RL/experi...,28
2,tabular_training_sarsa_1,/teamspace/studios/this_studio/Mario_RL/experi...,/teamspace/studios/this_studio/Mario_RL/experi...,17


In [12]:
# Optional: replace existing logs/log with recovered content for runs that currently have only the header.
# Set APPLY_RESTORE = True only if you want to overwrite logs/log.

APPLY_RESTORE = True

if APPLY_RESTORE:
    restored = []
    for r in results:
        target = r.recovered_log.with_name('log')
        target.write_text(r.recovered_log.read_text(encoding='utf-8'), encoding='utf-8')
        restored.append(str(target))

    print('Restored files:')
    for p in restored:
        print('-', p)
else:
    print('APPLY_RESTORE is False, existing logs/log files were not overwritten.')

Restored files:
- /teamspace/studios/this_studio/Mario_RL/experiments /deep_training/double_q_run_17_feb/logs/log
- /teamspace/studios/this_studio/Mario_RL/experiments /tabular_training/q_learning_17_feb/logs/log
- /teamspace/studios/this_studio/Mario_RL/experiments /tabular_training/tabular_training_sarsa_1/logs/log


In [20]:
# Rebuild reward/length/loss plots for every recovered log using MetricLogger.regenerate_plots_from_log()
import metric_logger
from importlib import reload
reload(metric_logger)

replot_summary = []
for r in results:
    recovered_path = r.recovered_log
    if not recovered_path.exists():
        continue

    logger = metric_logger.MetricLogger(save_dir=recovered_path.parent)
    logger.save_log = recovered_path
    logger.regenerate_plots_from_log()

    replot_summary.append({
        'run_name': r.run_name,
        'source_log': str(recovered_path),
        'reward_plot': str(logger.ep_rewards_plot),
        'length_plot': str(logger.ep_lengths_plot),
        'loss_plot': str(logger.ep_avg_losses_plot),
    })

pd.DataFrame(replot_summary)

,run_name,source_log,reward_plot,length_plot,loss_plot
0,double_q_run_17_feb,/teamspace/studios/this_studio/Mario_RL/experi...,/teamspace/studios/this_studio/Mario_RL/experi...,/teamspace/studios/this_studio/Mario_RL/experi...,/teamspace/studios/this_studio/Mario_RL/experi...
1,q_learning_17_feb,/teamspace/studios/this_studio/Mario_RL/experi...,/teamspace/studios/this_studio/Mario_RL/experi...,/teamspace/studios/this_studio/Mario_RL/experi...,/teamspace/studios/this_studio/Mario_RL/experi...
2,tabular_training_sarsa_1,/teamspace/studios/this_studio/Mario_RL/experi...,/teamspace/studios/this_studio/Mario_RL/experi...,/teamspace/studios/this_studio/Mario_RL/experi...,/teamspace/studios/this_studio/Mario_RL/experi...
